In [1]:
import pandas as pd

In [2]:
!ls ../data/

CustomerData_LeadGenerator.csv
sample_temperature_data_for_coding_challenge.csv


In [4]:
df1 = pd.read_csv('../data/CustomerData_LeadGenerator.csv')
df2 = pd.read_csv('../data/sample_temperature_data_for_coding_challenge.csv')

In [5]:
df1.head()

,fakeID,b_specialisation_i,b_specialisation_h,b_specialisation_g,b_specialisation_f,b_specialisation_e,b_specialisation_d,b_specialisation_c,b_specialisation_b,b_specialisation_a,...,q_5th Quint by Total HH,q_2017 Purchasing Power: Per Capita,q_2017 Total Population,"q_2017 Pop 15+/Edu: University, Fachhochschule",q_Uni by Total Pop,q_2017 Personal Care: Per Capita,q_2017 Medical Products: Per Capita,q_2017 Personal Effects: Per Capita,b_in_kontakt_gewesen,b_gekauft_gesamt
0,1069_7334,0,0,0,0,0,0,0,0,0,...,0.186735,23730.35,2194057,324656,0.147971,593.19,326.28,235.75,1,1
1,328_2350,0,0,0,1,1,1,0,0,0,...,0.179848,23343.26,1966798,281130,0.142938,588.54,320.36,230.66,1,1
2,865_6285,0,0,0,1,0,1,0,0,0,...,0.184935,23674.18,2149624,320326,0.149015,592.88,325.57,234.82,1,1
3,47_307,0,1,1,0,0,0,0,0,0,...,0.184523,23572.26,2161445,310981,0.143876,590.84,324.06,233.81,1,0
4,342_2513,0,0,0,0,0,1,0,0,0,...,0.189871,23821.63,2302959,334940,0.145439,593.56,327.75,237.41,1,1


In [8]:
df2.head()

,source_id,datetime,property_name,temperature
0,MICDEV001,2019-04-13T17:51:16.000+0000,heating_temperature,33.3
1,MICDEV001,2019-04-13T17:51:16.000+0000,cooling_temperature,15.0
2,MICDEV001,2019-04-13T18:51:18.000+0000,heating_temperature,34.0
3,MICDEV001,2019-04-13T19:51:20.000+0000,heating_temperature,33.8
4,MICDEV001,2019-04-13T20:51:21.000+0000,heating_temperature,34.2


In [11]:
df2.property_name.value_counts()

property_name
heating_temperature    699
cooling_temperature    301
Name: count, dtype: int64

In [13]:
df2.temperature.isna().sum()

0

In [18]:
df1.b_in_kontakt_gewesen.value_counts()

b_in_kontakt_gewesen
0    3678
1      95
Name: count, dtype: int64

In [17]:
df1.b_gekauft_gesamt.value_counts()

b_gekauft_gesamt
0    0.984893
1    0.015107
Name: proportion, dtype: float64

In [19]:
df1.columns

Index(['fakeID', 'b_specialisation_i', 'b_specialisation_h',
       'b_specialisation_g', 'b_specialisation_f', 'b_specialisation_e',
       'b_specialisation_d', 'b_specialisation_c', 'b_specialisation_b',
       'b_specialisation_a', 'b_specialisation_j', 'q_OpeningDays',
       'q_OpeningHours', 'q_2017 Average Household Size',
       'q_2017 Total Households',
       'q_2017 HHs: 5th Quintile (68.759 and above)',
       'q_5th Quint by Total HH', 'q_2017 Purchasing Power: Per Capita',
       'q_2017 Total Population',
       'q_2017 Pop 15+/Edu: University, Fachhochschule', 'q_Uni by Total Pop',
       'q_2017 Personal Care: Per Capita',
       'q_2017 Medical Products: Per Capita',
       'q_2017 Personal Effects: Per Capita', 'b_in_kontakt_gewesen',
       'b_gekauft_gesamt'],
      dtype='object')